In [447]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize

In [448]:
class FashionMNISTtask1(datasets.FashionMNIST):
  def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
    super(FashionMNISTtask1, self).__init__(root, train=train, transform=transform, target_transform=target_transform,download=download)
    self.classes = self.classes[:6]

  def __getitem__(self, index):
    img, target = super(FashionMNISTtask1, self).__getitem__(index)
    if target < 6:
        return img, target
    else:
        return img, -1


In [449]:
class FashionMNISTtask2(datasets.FashionMNIST):
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        super(FashionMNISTtask2, self).__init__(root, train=train, transform=transform, target_transform=target_transform,download=download)
        self.classes = self.classes[6:]

    def __getitem__(self, index):
        img, target = super(FashionMNISTtask2, self).__getitem__(index)
        if target >= 6:
            return img, target
        else:
            return img, -1

In [450]:
transform = torchvision.transforms.Compose([ToTensor(), Normalize((0.5), (0.5))])

train_dataset_task1 = FashionMNISTtask1(root='./data1', train=True, transform=transform, download=True)
test_dataset_task1 = FashionMNISTtask1(root='./data1', train=False, transform=transform, download=True)

In [451]:
train_dataset_task2 = FashionMNISTtask2(root='./data2', train=True, transform=transform, download=True)
test_dataset_task2 = FashionMNISTtask2(root='./data2', train=False, transform=transform, download=True)

In [452]:
train_dataset_filtered_task1 = [data for data in train_dataset_task1 if data[1] != -1]
test_dataset_filtered_task1 = [data for data in test_dataset_task1 if data[1] != -1]

train_dataset_filtered_task2 = [data for data in train_dataset_task2 if data[1] != -1]
test_dataset_filtered_task2 = [data for data in test_dataset_task2 if data[1] != -1]

In [453]:
train_dataloader_task1 = DataLoader(train_dataset_filtered_task1, batch_size=64, shuffle=True)
test_dataloader_task1 = DataLoader(test_dataset_filtered_task1, batch_size=256, shuffle=False)

for X, y in train_dataloader_task1:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"SHape of y: {y.shape}, dtype: {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
SHape of y: torch.Size([64]), dtype: torch.int64


In [454]:
len(train_dataloader_task1)

563

In [455]:
train_dataloader_task2 = DataLoader(train_dataset_filtered_task2, batch_size=64, shuffle=True)
test_dataloader_task2 = DataLoader(test_dataset_filtered_task2, batch_size=256, shuffle=False)

In [456]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [457]:
def kaiming_normal_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='sigmoid')

In [458]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes=10, hidden_size=512):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(28*28, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.classifier = nn.Linear(hidden_size, num_classes)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='sigmoid')
            elif isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        logits = self.classifier(x)

        return logits

In [459]:
class CNN(nn.Module):
    def __init__(self, num_classes=10, hidden_size=512):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32*14*14, hidden_size)
        self.relu3 = nn.ReLU()
        self.classifier = nn.Linear(hidden_size, num_classes)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='sigmoid')

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        logits = self.classifier(x)

        return logits

    def forward_features(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)

        return x


In [460]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    pred = model(X)
    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1) * len(X)
      print(f"Loss: {loss:>7f}, {current:>5d}/{size:>5d}")

In [461]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()

  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}, Avg Loss: {test_loss:>8f}\n")


In [462]:
model_task1 = CNN(num_classes=6, hidden_size=512).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_task1.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [463]:
model_task1.classifier.out_features

6

In [464]:
epochs = 3
for t in range(epochs):
  print(f"Epoch {t+1}\n---------------------------")
  train(train_dataloader_task1, model_task1, loss_fn, optimizer)
  test(test_dataloader_task1, model_task1, loss_fn)
print("Done!")

torch.save(model_task1.state_dict(), "model_old.pth")

Epoch 1
---------------------------
Loss: 1.847645,    64/36000
Loss: 0.213643,  6464/36000
Loss: 0.339809, 12864/36000
Loss: 0.223570, 19264/36000
Loss: 0.229604, 25664/36000
Loss: 0.233185, 32064/36000
Test Error: 
 Accuracy: 92.4, Avg Loss: 0.215037

Epoch 2
---------------------------
Loss: 0.165084,    64/36000
Loss: 0.117326,  6464/36000
Loss: 0.196695, 12864/36000
Loss: 0.146544, 19264/36000
Loss: 0.202948, 25664/36000
Loss: 0.152113, 32064/36000
Test Error: 
 Accuracy: 93.5, Avg Loss: 0.185432

Epoch 3
---------------------------
Loss: 0.152047,    64/36000
Loss: 0.114597,  6464/36000
Loss: 0.151217, 12864/36000
Loss: 0.077120, 19264/36000
Loss: 0.188129, 25664/36000
Loss: 0.113238, 32064/36000
Test Error: 
 Accuracy: 94.2, Avg Loss: 0.163278

Done!


In [465]:
class ModifiedCNN(CNN):
    def __init__(self, num_classes, hidden_size, new_hidden_size):
        super(ModifiedCNN, self).__init__(num_classes=num_classes, hidden_size=hidden_size)
        self.new_fc = nn.Linear(hidden_size, new_hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.classifier = nn.Linear(new_hidden_size, num_classes)

    def forward(self, x):
        x = super().forward_features(x)
        x = self.new_fc(x)
        x = self.relu(x)
        x = self.classifier(x)
        return x

In [466]:
model = ModifiedCNN(num_classes=10, hidden_size=512, new_hidden_size=512)

In [467]:
model

ModifiedCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=6272, out_features=512, bias=True)
  (relu3): ReLU()
  (classifier): Linear(in_features=512, out_features=10, bias=True)
  (new_fc): Linear(in_features=512, out_features=512, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)

# LWF

In [468]:
model_task2 = CNN(num_classes=6, hidden_size=512).to(device)
model_task1 = CNN(num_classes=6, hidden_size=512).to(device)

model_task2.load_state_dict(torch.load("model_old.pth"))
model_task1.load_state_dict(torch.load("model_old.pth"))

<All keys matched successfully>

In [469]:
model_task2.classifier.weight

Parameter containing:
tensor([[ 0.0782, -0.0211,  0.0398,  ..., -0.0276,  0.0631,  0.0041],
        [-0.1174,  0.0041,  0.0262,  ...,  0.1374, -0.1061, -0.0188],
        [ 0.1414,  0.0338,  0.0161,  ..., -0.0030,  0.1617, -0.0022],
        [ 0.0423,  0.0072, -0.0235,  ...,  0.0363, -0.0052,  0.1248],
        [-0.0654, -0.0065,  0.0216,  ..., -0.0733, -0.1650, -0.0572],
        [-0.0597, -0.0180, -0.0617,  ..., -0.0595, -0.0062,  0.1320]],
       requires_grad=True)

In [470]:
model_task2 = ModifiedCNN(num_classes=10, hidden_size=512, new_hidden_size=512)

In [471]:
model_task2.classifier.weight

Parameter containing:
tensor([[-0.0119, -0.0124, -0.0244,  ..., -0.0142, -0.0316,  0.0298],
        [-0.0142, -0.0385, -0.0383,  ...,  0.0369, -0.0402,  0.0102],
        [-0.0145, -0.0006, -0.0103,  ...,  0.0087, -0.0303, -0.0278],
        ...,
        [ 0.0023, -0.0003, -0.0191,  ..., -0.0118, -0.0129, -0.0430],
        [-0.0151,  0.0402,  0.0331,  ..., -0.0197, -0.0434,  0.0026],
        [-0.0111,  0.0158, -0.0247,  ...,  0.0414,  0.0152, -0.0147]],
       requires_grad=True)

In [472]:
in_features = model_task1.classifier.in_features
out_features = model_task1.classifier.out_features

new_weights = torch.zeros_like(model_task2.classifier.weight)
new_biases = torch.zeros_like(model_task2.classifier.bias)

weight = model_task1.classifier.weight.data
bias = model_task1.classifier.bias.data

new_weights[:out_features, :] = weight
new_biases[:out_features] = bias

In [473]:
model_task2.classifier.weight = nn.Parameter(new_weights)
model_task2.classifier.bias = nn.Parameter(new_biases)

In [474]:
model_task2.classifier.weight

Parameter containing:
tensor([[ 0.0782, -0.0211,  0.0398,  ..., -0.0276,  0.0631,  0.0041],
        [-0.1174,  0.0041,  0.0262,  ...,  0.1374, -0.1061, -0.0188],
        [ 0.1414,  0.0338,  0.0161,  ..., -0.0030,  0.1617, -0.0022],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       requires_grad=True)

In [475]:
model_task2 = model_task2.to(device)
print("New head numbers: ", model_task2.classifier.out_features)

New head numbers:  10


In [476]:
for param in model_task1.parameters():
  param.requires_grad = False

In [477]:
# Initially freeze all the layers except for classifier
for name, param in model_task2.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = False

Changes in training and testing

In [478]:
def train(alpha, T):
    size = len(train_dataloader_task2.dataset)
    # We set net_new to evaluation mode to prevent it from being updated
    # while computing the distillation loss from the old model

    model_task2.train()
    for batch, (X, y) in enumerate(train_dataloader_task2):
        X, y = X.to(device), y.to(device)

        outputs = model_task2(X)
        soft_y = model_task1(X)

        loss1 = loss_fn(outputs, y)

        outputs_S = nn.functional.softmax(outputs[:, :out_features] / T, dim=1)
        outputs_T = nn.functional.softmax(soft_y[:, :out_features] / T, dim=1)

        loss2 = outputs_T.mul(-1 * torch.log(outputs_S))
        loss2 = loss2.sum(1)
        loss2 = loss2.mean() * T * T

        loss = loss1 + alpha * loss2

        loss.backward(retain_graph=True)
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"Loss: {loss:>7f}, {current:>5d}/{size:>5d}")


In [479]:
def test(alpha, T):
  size = len(test_dataloader_task2.dataset)
  num_batches = len(test_dataloader_task2)
  model_task2.eval()

  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in test_dataloader_task2:
      X, y = X.to(device), y.to(device)

      outputs = model_task2(X)
      soft_y = model_task1(X)

      loss1 = loss_fn(outputs, y)

      outputs_S = nn.functional.softmax(outputs[:, :out_features] / T, dim=1)
      outputs_T = nn.functional.softmax(soft_y[:, :out_features] / T, dim=1)

      loss2 = outputs_T.mul(-1 * torch.log(outputs_S))
      loss2 = loss2.sum(1)
      loss2 = loss2.mean() * T * T

      loss = loss1 * alpha + loss2 * (1 - alpha)

      test_loss += loss.item()
      _, predicted = outputs.max(1)
      correct += predicted.eq(y).sum().item()
      # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}, Avg Loss: {test_loss:>8f}")


In [480]:
def val(epoch):
    model_task2.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(test_dataloader_task1):
            X, y = X.to(device), y.to(device)
            outputs = model_task2(X)
            _, predicted_old = outputs.max(1)
            total += len(y)
            correct += predicted_old.eq(y).sum().item()
        print(f"Validation Acc: {100. * correct / total}\n")

In [481]:
T = 2
alpha = 0.5
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model_task2.parameters(), lr=0.0001, momentum=0.9, weight_decay=5e-4)

for epoch in range(5):
    print(f"Epoch {epoch+1}: ----------------------")
    if epoch > 0:
        for param in model_task2.parameters():
            param.requires_grad = True
        optimizer = torch.optim.SGD(model_task2.parameters(), lr=0.0001, momentum=0.9)
    train(alpha, T)
    test(alpha, T)
    val(epoch)

Epoch 1: ----------------------
Loss: 5.941177,    64/24000
Loss: 5.392275,  6464/24000
Loss: 4.965646, 12864/24000
Loss: 4.605409, 19264/24000
Test Error: 
 Accuracy: 65.5, Avg Loss: 3.772757
Validation Acc: 1.85

Epoch 2: ----------------------
Loss: 4.622458,    64/24000
Loss: 3.544930,  6464/24000
Loss: 2.905811, 12864/24000
Loss: 2.399454, 19264/24000
Test Error: 
 Accuracy: 90.7, Avg Loss: 2.154190
Validation Acc: 0.0

Epoch 3: ----------------------
Loss: 2.484273,    64/24000
Loss: 2.261667,  6464/24000
Loss: 1.994310, 12864/24000
Loss: 2.213409, 19264/24000
Test Error: 
 Accuracy: 93.0, Avg Loss: 1.989996
Validation Acc: 0.0

Epoch 4: ----------------------
Loss: 1.847360,    64/24000
Loss: 2.076180,  6464/24000


KeyboardInterrupt: 